### 3 Mini-Golf

In [749]:
import robotic as ry
import random
import time
speed = 0.1

### Import 'mini_golf.g' and store home position

In [750]:
C = ry.Config()
print(ry.raiPath(''))
C.addFile(ry.raiPath('mini_golf.g'))

C.view()
qHomeJS = C.getJointState()
gripper_orig_pos = C.getFrame("l_gripper").getPosition()
gripper_orig_quat = C.getFrame("l_gripper").getQuaternion()
print(C.getFrameNames())

try:
  del bot
  del komo
except:
  pass

# initiliase bot and save home position
bot = ry.BotOp(C, useRealRobot=False)
qHome = bot.get_qHome()

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/robotic/rai-robotModels/
['world', 'world', 'table', 'l_panda_base', 'l_panda_link0', 'l_panda_link0_0', 'l_panda_joint1_origin', 'l_panda_joint1', 'l_panda_link1', 'l_panda_link1_0', 'l_panda_joint2_origin', 'l_panda_joint2', 'l_panda_link2', 'l_panda_link2_0', 'l_panda_joint3_origin', 'l_panda_joint3', 'l_panda_link3', 'l_panda_link3_0', 'l_panda_joint4_origin', 'l_panda_joint4', 'l_panda_link4', 'l_panda_link4_0', 'l_panda_joint5_origin', 'l_panda_joint5', 'l_panda_link5', 'l_panda_link5_0', 'l_panda_joint6_origin', 'l_panda_joint6', 'l_panda_link6', 'l_panda_link6_0', 'l_panda_joint7_origin', 'l_panda_joint7', 'l_panda_link7', 'l_panda_link7_0', 'l_panda_joint8_origin', 'l_panda_joint8', 'l_panda_link8', 'l_panda_hand_joint_origin', 'l_panda_hand_joint', 'l_panda_hand', 'l_panda_hand_0', 'l_panda_finger_joint1_origin', 'l_panda_finger_joint2_origin', 'l_panda_finger_joint1', 'l_panda_finger_joint2', 'l_panda_leftfing

### IK to move gripper to hammer handle

In [751]:
try:
  del komo
except:
  pass

komo = ry.KOMO(C, 1, 1, 0, False)
komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
# komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'hammer-handle'], ry.OT.eq, [1e1]) #constraint: gripper position

komo.addObjective(
    times=[],
    feature=ry.FS.positionRel,
    frames=['l_gripper', 'hammer-handle'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[0, 0., 0.015]
)

komo.addObjective(
    times=[],
    feature=ry.FS.scalarProductXX,
    frames=['l_gripper', 'hammer-handle'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[1]
)

komo.addObjective(
    times=[],
    feature=ry.FS.scalarProductYY,
    frames=['l_gripper', 'hammer-handle'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[1]
)

komo.addObjective(
    times=[],
    feature=ry.FS.scalarProductZZ,
    frames=['l_gripper', 'hammer-handle'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[1]
)

ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
    
q_hammer = komo.getPath()[0]
C.setJointState(q_hammer)
# C.attach('l_gripper', 'hammer-handle')
C.view()

0

### IK to move gripper for pre-hit

In [752]:
try:
  del komo
except:
  pass

komo = ry.KOMO(C, 1, 1, 0, False)
komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
# komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'hammer-handle'], ry.OT.eq, [1e1]) #constraint: gripper position

komo.addObjective(
    times=[],
    feature=ry.FS.positionRel,
    frames=['l_gripper', 'ball'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[-0.3, -0.3, 0.045]
)

komo.addObjective(
  times=[],
  feature=ry.FS.scalarProductXX,
  frames=['l_gripper', 'ball'],
  type=ry.OT.eq,
  scale=[1e1],
  target=[1]
)

komo.addObjective(
  times=[],
  feature=ry.FS.scalarProductYY,
  frames=['l_gripper', 'ball'],
  type=ry.OT.eq,
  scale=[1e1],
  target=[1]
)



ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
    
q_hit_pre = komo.getPath()[0]
C.setJointState(q_hit_pre)
C.view()

0

### IK to move gripper for hit

In [753]:
try:
  del komo
except:
  pass

komo = ry.KOMO(C, 1, 1, 0, False)
komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
# komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'hammer-handle'], ry.OT.eq, [1e1]) #constraint: gripper position

komo.addObjective(
    times=[],
    feature=ry.FS.positionRel,
    frames=['l_gripper', 'ball'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[0.3, -0.2, 0.04]
)

komo.addObjective(
  times=[],
  feature=ry.FS.vectorYDiff,
  frames=['l_gripper', 'ball'],
  type=ry.OT.eq,
  scale=[1e1],
  target=[0]
)

komo.addObjective(
  times=[],
  feature=ry.FS.scalarProductXX,
  frames=['l_gripper', 'ball'],
  type=ry.OT.eq,
  scale=[1e1],
  target=[1]
)

komo.addObjective(
  times=[],
  feature=ry.FS.scalarProductYY,
  frames=['l_gripper', 'ball'],
  type=ry.OT.eq,
  scale=[1e1],
  target=[1]
)

ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
    
q_hit = komo.getPath()[0]
C.setJointState(q_hit)
C.view()

0

### Move gripper to hammer handle

In [754]:
botOpSpeed = 2
bot.moveTo(q_hammer, timeCost=botOpSpeed)
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

### Grip the hammer handle

In [755]:
bot.gripperMove(ry._left, width=.01, speed=1)

while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

### Move ball to hole

In [756]:
# C.getFrame("ball").setPosition([ballHomePos[0], ballHomePos[1], ballHomePos[2]])

bot.moveTo(q_hit_pre, timeCost=botOpSpeed)
bot.moveTo(q_hit, timeCost=botOpSpeed)
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

### Place hammer back

In [757]:
# Reset bot
bot.moveTo(q_hit_pre, timeCost=botOpSpeed)
bot.moveTo(q_hammer, timeCost=botOpSpeed)

while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

### Return to home position

In [758]:
bot.gripperMove(ry._left, width=0.075, speed=1)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)
bot.moveTo(qHome, timeCost=botOpSpeed)
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [760]:
del bot

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
